In [1]:
"""
Collaborative Filtering ALS Recommender System using Spark MLlib adapted from
the Spark Summit 2014 Recommender System training example.

Developed By: Pranav Masariya
Supervisor: Dr. Magdalini Eirinaki
"""

import os
import numpy as np
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from pyspark.mllib.recommendation import ALS
from pyspark.ml.recommendation import ALS as mlals
from pyspark.ml.evaluation import RegressionEvaluator

import math
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark.ml.evaluation import RegressionEvaluator

In [2]:
# Calling spark session to register application
spark = SparkSession \
    .builder \
    .appName("Recom") \
    .config("spark.recom.demo", "1") \
    .getOrCreate()

In [3]:
"""
Loading and Parsing Dataset
    Each line in the ratings dataset (ratings.csv) is formatted as:
         userId,movieId,rating,timestamp
    Each line in the movies (movies.csv) dataset is formatted as:
        movieId,title,genres

""" 

# Load ratings
ratings_df = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("ratings.csv")

In [4]:
ratings_df

DataFrame[userId: int, movieId: int, rating: double]

In [5]:
"""
For the simplicity of this tutorial
    For each line in the ratings dataset, we create a tuple of (UserID, MovieID, Rating). 
    We drop the timestamp because we do not need it for this recommender.
"""

#ratings_df = ratings_df.drop('timestamp')
ratings_df.show(3)

+------+-------+------------+
|userId|movieId|      rating|
+------+-------+------------+
|   149|      2| 0.927561837|
|   149|      3|-0.839222615|
|   149|      5|-0.839222615|
+------+-------+------------+
only showing top 3 rows



In [6]:
"""
In order to determine the best ALS parameters, we will use the small dataset. 
We need first to split it into train, validation, and test datasets.
"""
(trainingData,validationData,testData) = ratings_df.randomSplit([0.8,0.1,0.1])

In [7]:
# Prepare test and validation set. They should not have ratings

validation_for_predict = validationData.select('userId','movieId')
test_for_predict = testData.select('userId','movieId')

In [8]:
"""
Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by 
using Alternating Least Squares. The implementation in MLlib has the following parameters:

    1. numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
    2. rank is the number of latent factors in the model.
    3. iterations is the number of iterations to run.
    4. lambda specifies the regularization parameter in ALS.
    5. implicitPrefs specifies whether to use the explicit 
        feedback ALS variant or one adapted for implicit feedback data.
    6. alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline 
        confidence in preference observations.

"""

seed = 5 #Random seed for initial matrix factorization model. A value of None will use system time as the seed.
iterations = 10
regularization_parameter = 0.1 #run for different lambdas - e.g. 0.01
ranks = [4, 8, 12] #number of features
errors = [0, 0, 0]
err = 0
tolerance = 0.02

min_error = float('inf')
best_rank = -1
best_iteration = -1

In [9]:
# Let us traing our dataset and check the best rank with lowest RMSE
# predictAll method of the ALS takes only RDD format and hence we need to convert our dataframe into RDD
# df.rdd will automatically converts Dataframe into RDD

for rank in ranks:
    model = ALS.train(trainingData, rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
    predictions = model.predictAll(validation_for_predict.rdd).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = validationData.rdd.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean()) # RMSE Error
    errors[err] = error
    err += 1
    print ('For rank %s the RMSE is %s' % (rank, error))
    if error < min_error:
        min_error = error
        best_rank = rank

print ('The best model was trained with rank %s' % best_rank)

For rank 4 the RMSE is 1.0990339415
For rank 8 the RMSE is 0.965288738897
For rank 12 the RMSE is 0.927297201283
The best model was trained with rank 12


In [10]:
"""
Spark will soon deprecate MLLIb package. 
They are focusing more on ML packages with standard machine learning implementation
Let's see that package also
"""
als =  mlals(maxIter=iterations,rank=4,seed=seed,regParam=regularization_parameter, userCol="userId", itemCol="movieId",ratingCol="rating")
modelML = als.fit(trainingData)
pred = modelML.transform(validationData)
pred = pred.where(pred['prediction'] != 'NaN')
    
# Evaluate the model by computing RMSE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(pred)

print ('RMSE is %s' % rmse)

"""
The best part is we do not have to worry about RDD any more with this library
"""

RMSE is 1.07152402993


'\nThe best part is we do not have to worry about RDD any more with this library\n'

In [11]:
# Let's take test dataset and get ratings
predictions_test = model.predictAll(test_for_predict.rdd).map(lambda r: ((r[0], r[1]), r[2]))

In [12]:
## visualize preditions, here third element is predictions generated by ALS Model
predictions_test.take(3)

[((460, 5), 0.14467233353957915),
 ((460, 14), -0.18127298287070573),
 ((912, 16), -0.8881264110490364)]

In [13]:
"""
Let's start recommending movies.
I have written a method to call recommendations for a perticular user from test data

TODO: You need to execute one more step before calling getRecommendations, 
      Think about that step. If you go through the seps below, you will realize it soon.
"""
def getRecommendations(user,testDf,trainDf,model):
    # get all user and his/her rated movies
    userDf = testDf.filter(testDf.userId == user)
    # filter movies from main set which have not been rated by selected user
    # and pass it to model we sreated above
    mov = trainDf.select('movieId').subtract(userDf.select('movieId'))
    
    # Again we need to covert our dataframe into RDD
    pred_rat = model.predictAll(mov.rdd.map(lambda x: (user, x[0]))).collect()
    
    # Get the top recommendations
    recommendations = sorted(pred_rat, key=lambda x: x[2], reverse=True)[:50]
    
    return recommendations

In [14]:
# Assign user id for which we need recommendations#user = 337
user = 969

# Call getRecommendations method
derived_rec = getRecommendations(user,testData,trainingData,model)

print ("Movies recommended for:%d" % user)



Movies recommended for:969


In [16]:
derived_rec

[Rating(user=969, product=2, rating=0.028894738022447047),
 Rating(user=969, product=20, rating=0.01744645490047858),
 Rating(user=969, product=12, rating=-0.012485262634038707),
 Rating(user=969, product=6, rating=-0.024314851056819187),
 Rating(user=969, product=18, rating=-0.17090410571053893),
 Rating(user=969, product=8, rating=-0.3109935539096452),
 Rating(user=969, product=7, rating=-0.4136548171532802),
 Rating(user=969, product=1, rating=-0.45489023824419766),
 Rating(user=969, product=19, rating=-0.4800760295483392),
 Rating(user=969, product=13, rating=-0.6949371250882268),
 Rating(user=969, product=4, rating=-0.7103406776721104),
 Rating(user=969, product=10, rating=-0.8980401526672288),
 Rating(user=969, product=3, rating=-0.9277905747328498),
 Rating(user=969, product=9, rating=-1.0301495058540413),
 Rating(user=969, product=21, rating=-1.1122173966532916),
 Rating(user=969, product=11, rating=-2.0662874865403187),
 Rating(user=969, product=15, rating=-2.10287766524094),


In [15]:
for i in range(5):
    print (derived_rec[i])

Rating(user=969, product=2, rating=0.028894738022447047)
Rating(user=969, product=20, rating=0.01744645490047858)
Rating(user=969, product=12, rating=-0.012485262634038707)
Rating(user=969, product=6, rating=-0.024314851056819187)
Rating(user=969, product=18, rating=-0.17090410571053893)
